## Load data

In [1]:
import pandas as pd

url_univdorm= 'https://raw.githubusercontent.com/irenekarijadi/RF-LSTM-CEEMDAN/main/Dataset/data%20of%20UnivDorm_Prince.csv'
univdorm= pd.read_csv(url_univdorm)
data_univdorm= univdorm[(univdorm['timestamp'] > '2015-03-01') & (univdorm['timestamp'] < '2015-06-01')]
dfs_univdorm=data_univdorm['energy']
datas_univdorm=pd.DataFrame(dfs_univdorm)


## import libraries

In [2]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

from PyEMD import CEEMDAN
import numpy
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn import metrics

import time
import dataframe_image as dfi


## Import all functions from another notebook for building prediction methods

In [3]:
import Setting
from myfunctions import lr_model,svr_model,ann_model,rf_model,lstm_model,hybrid_ceemdan_rf,hybrid_ceemdan_lstm,proposed_method

## Import parameter settings

In [4]:
hours=Setting.n_hours
data_partition=Setting.data_partition
max_features=Setting.max_features
epoch=Setting.epoch
batch_size=Setting.batch_size
neuron=Setting.neuron
lr=Setting.lr
optimizer=Setting.optimizer

## Run the experiments
### Run this following cell will train and test the proposed method and other benchmark methods on University Dormitory Dataset

In [5]:
#Linear Regression

start_time = time.time()
lr_univdorm=lr_model(datas_univdorm,hours,data_partition)
lr_time_univdorm=time.time() - start_time
print("--- %s seconds - Linear Regression- univdorm ---" % (lr_time_univdorm))

#Support Vector Regression
start_time = time.time()
svr_univdorm=svr_model(datas_univdorm,hours,data_partition)
svr_time_univdorm=time.time() - start_time
print("--- %s seconds - Support Vector Regression- univdorm ---" % (svr_time_univdorm))


#ANN
start_time = time.time()
ann_univdorm=ann_model(datas_univdorm,hours,data_partition)
ann_time_univdorm=time.time() - start_time
print("--- %s seconds - ANN- univdorm ---" % (ann_time_univdorm))

#random forest
start_time = time.time()
rf_univdorm=rf_model(datas_univdorm,hours,data_partition,max_features)
rf_time_univdorm=time.time() - start_time
print("--- %s seconds - Random Forest- univdorm ---" % (rf_time_univdorm))

#LSTM
start_time = time.time()
lstm_univdorm=lstm_model(datas_univdorm,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
lstm_time_univdorm=time.time() - start_time
print("--- %s seconds - lstm- univdorm ---" % (lstm_time_univdorm))


#CEEMDAN RF
start_time = time.time()
ceemdan_rf_univdorm=hybrid_ceemdan_rf(dfs_univdorm,hours,data_partition,max_features)
ceemdan_rf_time_univdorm=time.time() - start_time
print("--- %s seconds - ceemdan_rf- univdorm ---" % (ceemdan_rf_time_univdorm))

#CEEMDAN LSTM
start_time = time.time()
ceemdan_lstm_univdorm=hybrid_ceemdan_lstm(dfs_univdorm,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
ceemdan_lstm_time_univdorm=time.time() - start_time
print("--- %s seconds - ceemdan_lstm- univdorm ---" % (ceemdan_lstm_time_univdorm))


#proposed method
start_time = time.time()
proposed_method_univdorm=proposed_method(dfs_univdorm,hours,data_partition,max_features,epoch,batch_size,neuron,lr,optimizer)
proposed_method_time_univdorm=time.time() - start_time
print("--- %s seconds - proposed_method- univdorm ---" % (proposed_method_time_univdorm))



--- 0.204878568649292 seconds - Linear Regression- univdorm ---
--- 0.31400203704833984 seconds - Support Vector Regression- univdorm ---
--- 1.5430538654327393 seconds - ANN- univdorm ---
--- 1.160142183303833 seconds - Random Forest- univdorm ---
--- 6.8299877643585205 seconds - lstm- univdorm ---
--- 31.396042346954346 seconds - ceemdan_rf- univdorm ---
--- 90.53752374649048 seconds - ceemdan_lstm- univdorm ---
--- 83.12702703475952 seconds - proposed_method- univdorm ---


## Summarize of experimental results with running time
### Run this following cell will summarize the result and generate output used in Section 4.4 (Table 3) for University Dormitory dataset

In [6]:
running_time_univdorm=pd.DataFrame([lr_time_univdorm,svr_time_univdorm,ann_time_univdorm,
                                   rf_time_univdorm,lstm_time_univdorm,ceemdan_rf_time_univdorm,
                                   ceemdan_lstm_time_univdorm,proposed_method_time_univdorm])
running_time_univdorm=running_time_univdorm.T
running_time_univdorm.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']
proposed_method_univdorm_df=proposed_method_univdorm[0:3]
result_univdorm=pd.DataFrame([lr_univdorm,svr_univdorm,ann_univdorm,rf_univdorm,lstm_univdorm,ceemdan_rf_univdorm,
                    ceemdan_lstm_univdorm,proposed_method_univdorm_df])
result_univdorm=result_univdorm.T
result_univdorm.columns=['LR','SVR','ANN','RF','LSTM','CEEMDAN RF','CEEMDAN LSTM','Proposed Method']
univdorm_summary=pd.concat([result_univdorm,running_time_univdorm],axis=0)

univdorm_summary.set_axis(['MAPE(%)', 'RMSE','MAE','running time (s)'], axis='index')

univdorm_summary.style.set_caption("University Dormitory Results")
index = univdorm_summary.index
index.name = "university dormitory results"
univdorm_summary

,LR,SVR,ANN,RF,LSTM,CEEMDAN RF,CEEMDAN LSTM,Proposed Method
university dormitory results,,,,,,,,
MAPE(%),6.096747,6.486764,6.719073,6.144907,6.829237,3.955034,4.080925,3.511434
RMSE,3.091409,3.386055,3.389476,3.172311,3.468066,2.007754,2.069735,1.761154
MAE,2.401101,2.590120,2.640636,2.435938,2.677671,1.550750,1.586834,1.369341
running time (s),0.204879,0.314002,1.543054,1.160142,6.829988,31.396042,90.537524,83.127027


In [7]:
#export table to png
#dfi.export(univdorm_summary,"univdorm_summary_table.png")

## Calculate percentage improvement
### Run this following cell will calculate percentage improvement and generate output used in Section 4.4 (Table 4) for University Dormitory dataset

In [8]:
pMAPE_LR_vs_Proposed_univdorm=((lr_univdorm[0]-proposed_method_univdorm[0])/lr_univdorm[0])*100
pRMSE_LR_vs_Proposed_univdorm=((lr_univdorm[1]-proposed_method_univdorm[1])/lr_univdorm[1])*100
pMAE_LR_vs_Proposed_univdorm=((lr_univdorm[2]-proposed_method_univdorm[2])/lr_univdorm[2])*100

pMAPE_SVR_vs_Proposed_univdorm=((svr_univdorm[0]-proposed_method_univdorm[0])/svr_univdorm[0])*100
pRMSE_SVR_vs_Proposed_univdorm=((svr_univdorm[1]-proposed_method_univdorm[1])/svr_univdorm[1])*100
pMAE_SVR_vs_Proposed_univdorm=((svr_univdorm[2]-proposed_method_univdorm[2])/svr_univdorm[2])*100

pMAPE_ANN_vs_Proposed_univdorm=((ann_univdorm[0]-proposed_method_univdorm[0])/ann_univdorm[0])*100
pRMSE_ANN_vs_Proposed_univdorm=((ann_univdorm[1]-proposed_method_univdorm[1])/ann_univdorm[1])*100
pMAE_ANN_vs_Proposed_univdorm=((ann_univdorm[2]-proposed_method_univdorm[2])/ann_univdorm[2])*100

pMAPE_RF_vs_Proposed_univdorm=((rf_univdorm[0]-proposed_method_univdorm[0])/rf_univdorm[0])*100
pRMSE_RF_vs_Proposed_univdorm=((rf_univdorm[1]-proposed_method_univdorm[1])/rf_univdorm[1])*100
pMAE_RF_vs_Proposed_univdorm=((rf_univdorm[2]-proposed_method_univdorm[2])/rf_univdorm[2])*100

pMAPE_LSTM_vs_Proposed_univdorm=((lstm_univdorm[0]-proposed_method_univdorm[0])/lstm_univdorm[0])*100
pRMSE_LSTM_vs_Proposed_univdorm=((lstm_univdorm[1]-proposed_method_univdorm[1])/lstm_univdorm[1])*100
pMAE_LSTM_vs_Proposed_univdorm=((lstm_univdorm[2]-proposed_method_univdorm[2])/lstm_univdorm[2])*100

pMAPE_ceemdan_rf_vs_Proposed_univdorm=((ceemdan_rf_univdorm[0]-proposed_method_univdorm[0])/ceemdan_rf_univdorm[0])*100
pRMSE_ceemdan_rf_vs_Proposed_univdorm=((ceemdan_rf_univdorm[1]-proposed_method_univdorm[1])/ceemdan_rf_univdorm[1])*100
pMAE_ceemdan_rf_vs_Proposed_univdorm=((ceemdan_rf_univdorm[2]-proposed_method_univdorm[2])/ceemdan_rf_univdorm[2])*100


pMAPE_ceemdan_lstm_vs_Proposed_univdorm=((ceemdan_lstm_univdorm[0]-proposed_method_univdorm[0])/ceemdan_lstm_univdorm[0])*100
pRMSE_ceemdan_lstm_vs_Proposed_univdorm=((ceemdan_lstm_univdorm[1]-proposed_method_univdorm[1])/ceemdan_lstm_univdorm[1])*100
pMAE_ceemdan_lstm_vs_Proposed_univdorm=((ceemdan_lstm_univdorm[2]-proposed_method_univdorm[2])/ceemdan_lstm_univdorm[2])*100


df_PI_univdorm=[[pMAPE_LR_vs_Proposed_univdorm,pMAPE_SVR_vs_Proposed_univdorm,pMAPE_ANN_vs_Proposed_univdorm,
                pMAPE_RF_vs_Proposed_univdorm,pMAPE_LSTM_vs_Proposed_univdorm,pMAPE_ceemdan_rf_vs_Proposed_univdorm,
                pMAPE_ceemdan_lstm_vs_Proposed_univdorm],
                [pRMSE_LR_vs_Proposed_univdorm,pRMSE_SVR_vs_Proposed_univdorm,pRMSE_ANN_vs_Proposed_univdorm,
                pRMSE_RF_vs_Proposed_univdorm,pRMSE_LSTM_vs_Proposed_univdorm,pRMSE_ceemdan_rf_vs_Proposed_univdorm,
                pRMSE_ceemdan_lstm_vs_Proposed_univdorm],
                [pMAE_LR_vs_Proposed_univdorm,pMAE_SVR_vs_Proposed_univdorm,pMAE_ANN_vs_Proposed_univdorm,
                pMAE_RF_vs_Proposed_univdorm,pMAE_LSTM_vs_Proposed_univdorm,pMAE_ceemdan_rf_vs_Proposed_univdorm,
                pMAE_ceemdan_lstm_vs_Proposed_univdorm]]

PI_univdorm=pd.DataFrame(df_PI_univdorm, columns=["LR vs. Proposed Method", "SVR vs. Proposed Method","ANN vs. Proposed Method",
                                      "RF vs. Proposed Method","LSTM vs. Proposed Method","CEEMDAN RF vs. Proposed Method",
                                      "CEEMDAN RF LSTM vs. Proposed Method"])
PI_univdorm= PI_univdorm.round(decimals = 2)
PI_univdorm.set_axis(['MAPE(%)', 'RMSE','MAE'], axis='index')
PI_univdorm.style.set_caption("Percentage Improvement-University Dormitory Building")
index = PI_univdorm.index
index.name = "Percentage Improvement university dormitory"
PI_univdorm

,LR vs. Proposed Method,SVR vs. Proposed Method,ANN vs. Proposed Method,RF vs. Proposed Method,LSTM vs. Proposed Method,CEEMDAN RF vs. Proposed Method,CEEMDAN RF LSTM vs. Proposed Method
Percentage Improvement university dormitory,,,,,,,
MAPE(%),42.40,45.87,47.74,42.86,48.58,11.22,13.95
RMSE,43.03,47.99,48.04,44.48,49.22,12.28,14.91
MAE,42.97,47.13,48.14,43.79,48.86,11.70,13.71


In [9]:
#export table to png
#dfi.export(PI_univdorm,"PI_univdorm_table.png")